<!--TITLE:Custom Convnets-->
# Introduction #

Now that you've seen the layers a convnet uses to extract features, it's time to put them together and build a network of your own!

# Simple to Complex #

More extraction makes better features.

# Convolutional Blocks #

Arrange them into blocks.

# Example - Design a Convnet #

## Step 1 ##

In [ ]:
#$HIDE_INPUT$
# Imports
import os
import warnings
import numpy as np
import visiontools
from visiontools import StanfordCars
import tensorflow as tf
import tensorflow_datasets as tfds

# Reproducibility
def seed_everything(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed = 31415
seed_everything(seed)
warnings.filterwarnings("ignore")

# Load training and validation sets
DATA_DIR = '/kaggle/input/stanford-cars-for-learn/'
(ds_train_, ds_valid_), ds_info = tfds.load(
    'stanford_cars/simple',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
    data_dir=DATA_DIR,
)
print(("Loaded {} training examples " +
       "and {} validation examples " +
       "with classes {}.").format(
           ds_info.splits['train'].num_examples,
           ds_info.splits['test'].num_examples,
           ds_info.features['label'].names))

# Create data pipeline
BATCH_SIZE = 16
AUTO = tf.data.experimental.AUTOTUNE
SIZE = [192, 192]
preprocess = visiontools.make_preprocessor(size=SIZE)

ds_train = (ds_train_
            .map(preprocess)
            .cache()
            .shuffle(ds_info.splits['train'].num_examples)
            .map(augment, AUTO)            
            .batch(BATCH_SIZE)
            .prefetch(AUTO))

ds_valid = (ds_valid_
            .map(preprocess)
            .cache()
            .shuffle(ds_info.splits['test'].num_examples)
            .batch(BATCH_SIZE)
            .prefetch(AUTO))

## Step 2 ##

In [ ]:
model = keras.Sequential([

    # First Convolutional Block
    layers.Conv2D(filters=64,
                  kernel_size=5,
                  activation="relu",
                  padding='same')
    layers.MaxPool2D(),

    # Second Convolutional Block    
    layers.Conv2D(filters=128,
                  kernel_size=3,
                  activation="relu",
                  padding='same')
    layers.MaxPool2D(),

    # Third Convolutional Block    
    layers.Conv2D(filters=256,
                  kernel_size=3,
                  activation="relu",
                  padding='same')
    layers.MaxPool2D(),

    # Classifier Head #
    layers.Flatten(),
    layers.Dense(units=8,
                 activation="relu"),
    layers.Dense(units=1,
                 activation="sigmoid"),
])

## Step 3 ##

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)

history = model.fit(ds_train_,
                    validation_data=ds_valid_,
                    epochs=10)

In [ ]:
import pandas as pd

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot();

# Conclusion #